In [1]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models


data_dir = r'C:\Users\asha\project2\flowers'  
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

flower_types = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

for flower in flower_types:
    flower_dir = os.path.join(data_dir, flower)


    if os.path.isdir(flower_dir):
        images = os.listdir(flower_dir)

        
        random.shuffle(images)
        split_index = int(len(images) * 0.8) 
        train_images = images[:split_index]
        val_images = images[split_index:]

 
        train_flower_dir = os.path.join(train_dir, flower)
        os.makedirs(train_flower_dir, exist_ok=True)

        for image in train_images:
            src_path = os.path.join(flower_dir, image)
            dst_path = os.path.join(train_flower_dir, image)
            if os.path.exists(src_path):  
                try:
                    shutil.copy(src_path, dst_path)
                except PermissionError:
                    print(f"permison error: {src_path} can't copy.")
                except Exception as e:
                    print(f"error created: {str(e)}")
            else:
                print(f"not excist: {src_path}")

     
        val_flower_dir = os.path.join(val_dir, flower)
        os.makedirs(val_flower_dir, exist_ok=True)

        for image in val_images:
            src_path = os.path.join(flower_dir, image)
            dst_path = os.path.join(val_flower_dir, image)
            if os.path.exists(src_path): 
                try:
                    shutil.copy(src_path, dst_path)
                except PermissionError:
                    print(f"permison error: {src_path} can't copy.")
                except Exception as e:
                    print(f"error created: {str(e)}")
            else:
                print(f"not excist: {src_path}")

print("Images successfully separated into train and validation folders!")



print("Train:")
for flower in os.listdir(train_dir):
    flower_path = os.path.join(train_dir, flower)
    print(f" - {flower}: {len(os.listdir(flower_path))} image")


print("\nValidation:")
for flower in os.listdir(val_dir):
    flower_path = os.path.join(val_dir, flower)
    print(f" - {flower}: {len(os.listdir(flower_path))} image")



train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical'  
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False 

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(len(flower_types), activation='softmax'))  


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=10
)


model.save('flower_classifier_model.h5')

model.summary()


Images successfully separated into train and validation folders!
Train:
 - daisy: 764 image
 - dandelion: 1052 image
 - rose: 784 image
 - sunflower: 733 image
 - tulip: 984 image

Validation:
 - daisy: 739 image
 - dandelion: 1019 image
 - rose: 748 image
 - sunflower: 704 image
 - tulip: 941 image
Found 4317 images belonging to 5 classes.
Found 4151 images belonging to 5 classes.


D:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 1126s 8s/step - accuracy: 0.2778 - loss: 6.1672 - val_accuracy: 0.3580 - val_loss: 1.5400
Epoch 2/10
  1/134 ━━━━━━━━━━━━━━━━━━━━ 8:45 4s/step - accuracy: 0.4062 - loss: 1.6482

D:\anaconda\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.4062 - loss: 1.6482 - val_accuracy: 0.3043 - val_loss: 1.5248
Epoch 3/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 994s 7s/step - accuracy: 0.4468 - loss: 1.3866 - val_accuracy: 0.4840 - val_loss: 1.2577
Epoch 4/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.4375 - loss: 1.2294 - val_accuracy: 0.4783 - val_loss: 1.4649
Epoch 5/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 952s 7s/step - accuracy: 0.5020 - loss: 1.2345 - val_accuracy: 0.5155 - val_loss: 1.2231
Epoch 6/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.5938 - loss: 0.9091 - val_accuracy: 0.5652 - val_loss: 1.4691
Epoch 7/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 3288s 25s/step - accuracy: 0.5009 - loss: 1.2854 - val_accuracy: 0.5848 - val_loss: 1.0450
Epoch 8/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6250 - loss: 1.1158 - val_accuracy: 0.5217 - val_loss: 1.2073
Epoch 9/10
134/134 ━━━━━━━━━━━━━━━━━━━━ 2106s 16s/step - accuracy: 0.5436 - loss: 1.1670 - val_accuracy: 0

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,662,673 (384.00 MB)

 Trainable params: 25,691,653 (98.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 51,383,308 (196.01 MB)